<a href="https://colab.research.google.com/github/asigalov61/Amazing-GPT2-Piano/blob/master/Amazing_GPT2_Piano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AMAZING GPT2 PIANO

MAKE YOUR OWN SOTA MUSIC AI MODEL IN UNDER 10 MINUTES :)

This is a slightly modified fork of Park Soochul Google Colab Notebook from his Music-GPT2 repo https://github.com/scpark20/Music-GPT-2

MAESTRO Dataset is courtesy of Google Magenta Team and it is distributed under Attribution-NonCommercial-ShareAlike 4.0 International license. 

So keep this in mind and respect everyone's copyright, please :)

Huge thanks go out to Ravi A. for his invaluable consultations and contributions that made this Colab notebook possible :) Thank you so much, bro!

# I. Environment Setup and DataSet Prep

In [ ]:
#@title Install all requirements/dependencies. You may have to run this section twice or trice at first run and you may have to restart the runtime/see version errors. It is normal. However, please report all other issues/critical errors on my GitHub so that I can fix it. This section may take a while to execute. Please be patient. Thank you :)
print('3...2...1...lets go...')
%tensorflow_version 1.x
!pip install --upgrade tensorflow-gpu==1.14
!pip install --upgrade tensorboard==1.15.0
!pip install --upgrade tensorboardX
!pip install --upgrade tensorflow-estimator==1.15.0
!pip install --upgrade numpy
!pip install --upgrade librosa
!pip install --upgrade gast==0.2.2
!pip install --upgrade tqdm
!apt-get install --upgrade swig
!pip install --upgrade git+https://github.com/asigalov61/python3-midi
!pip install --upgrade pypianoroll
!pip install --upgrade pretty-midi
#print('Copying checkpoints and Salamander piano SoundFont (via https://sites.google.com/site/soundfonts4u) from GCS...')
#!gsutil -q -m cp gs://magentadata/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /content/
#print('Installing nice MIDI rendering code...')
#!apt-get update -qq && apt-get install -qq libfluidsynth1 build-essential libasound2-dev libjack-dev
#!pip install -qU pyfluidsynth

#import ctypes.util
#def proxy_find_library(lib):
#  if lib == 'fluidsynth':
#    return 'libfluidsynth.so.1'
#  else:
#    return ctypes.util.find_library(lib)
#ctypes.util.find_library = proxy_find_library
!nvidia-smi
print('Success :) Everything is installed and should work fine :) Enjoy!')

#### Data Files/Paths Preparation

Download and unzip Training MIDIs Dataset of your choice :)

In [ ]:
#@title (Preffered) Download and Unzip Alex Piano MIDI Dataset (~2000 MIDIs)
!wget 'https://github.com/asigalov61/AlexMIDIDataSet/raw/master/AlexMIDIDataSet-CC-BY-NC-SA-Piano-Only.zip'
!wget 'https://github.com/asigalov61/AlexMIDIDataSet/raw/master/AlexMIDIDataSet-CC-BY-NC-SA-All-Drafts-Piano-Only.zip'
!unzip -j 'AlexMIDIDataSet-CC-BY-NC-SA-Piano-Only.zip'
!unzip -j 'AlexMIDIDataSet-CC-BY-NC-SA-All-Drafts-Piano-Only.zip'

In [ ]:
#@title (Optional) Download and Unzip Google Magenta MAESTRO Piano MIDI Dataset (~1300 MIDIs)
!wget 'https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip'
!unzip -j /content/maestro-v2.0.0-midi.zip

Calculate the total number of available and indexable MIDI files

In [ ]:
#@title Calculate Number of Indexable MIDI files :)
import os
from os import listdir
from os.path import isfile, join

# Modify these paths to your own paths to MAESTRO dataset
maestro_dir = '/content'

data_dirs = ['/content']

data_files = []
for data_dir in data_dirs:
    data_files += [join(data_dir, f) for f in listdir(data_dir) if isfile(join(data_dir, f)) if 'mid' or 'midi' in f] # MIDI files can have MIDI and MID extensions. Remove/comment the one you do not need/use

data_files.sort()

print('total midi/mid files : ', len(data_files))

### Event Extract from Midi Files

In [ ]:
#@title Extract Events from MIDI files
import mido
import numpy as np


def get_eventlist(data_file):
    ON = 1
    OFF = 0
    CC = 2

    midi = mido.MidiFile(data_file)

    current_time = 0
    eventlist = []
    cc = False
    for msg in midi:
        #print(msg)
        current_time += msg.time

         # NOTE ON CASE
        if msg.type is 'note_on' and msg.velocity > 0:
            event = [current_time, ON, msg.note, msg.velocity]
            eventlist.append(event)

         # NOTE OFF CASE        
        elif msg.type is 'note_off' or (msg.type is 'note_on' and msg.velocity == 0):
            event = [current_time, OFF, msg.note, msg.velocity]
            eventlist.append(event)
            
        if msg.type is 'control_change':
            
            if msg.control != 64:
                continue
            
            if cc == False and msg.value > 0:
                cc = True
                event = [current_time, CC, 0, 1]
                eventlist.append(event)
                
            elif cc == True and msg.value == 0:
                cc = False
                event = [current_time, CC, 0, 0]
                eventlist.append(event)
                
    eventlist = np.array(eventlist)
    return eventlist

index = np.random.randint(0, len(data_files))
print(index)
eventlist = get_eventlist(data_files[index])
print(eventlist)

### Midifile to EventListfile

In [ ]:
#@title Convert each Midifile to EventListfile
from tqdm import tqdm_notebook as tqdm
import os

dataset_dir = 'dataset'

if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

for i in tqdm(range(len(data_files))):
    print(data_files[i])
    eventlist = get_eventlist(data_files[i])
    print(eventlist.shape)
    
    save_file = dataset_dir + '/' + str(i)
    data = {'eventlist': eventlist}
    np.savez(save_file, **data, allow_pickle=False)

# II. Training and Generating Output

### Data List

In [ ]:
#@title Create a Data List from MIDI files
import os
from os import listdir
from os.path import isfile, join
import ntpath

data_dirs = ['dataset']

data_files = []

for data_dir in data_dirs:
    data_files += [join(data_dir, f) for f in listdir(data_dir) if isfile(join(data_dir, f)) if '.npz' in f]
print(len(data_files))

data_files.sort()

### Hyperparameters Setting

In [ ]:
#@title Setup the GPT2 Model (Hyperparameters for the model structure)
number_of_layers = 8 #@param {type:"slider", min:1, max:16, step:1}
number_of_heads = 4 #@param {type:"slider", min:1, max:32, step:1}
embeding_size = 256 #@param {type:"slider", min:0, max:1024, step:128}
constant_time_length = 4096 #@param {type:"slider", min:0, max:8192, step:128}
IntervalDim = 100

VelocityDim = 32
VelocityOffset = IntervalDim

NoteOnDim = NoteOffDim = 128
NoteOnOffset = IntervalDim + VelocityDim
NoteOffOffset = IntervalDim + VelocityDim + NoteOnDim

CCDim = 2
CCOffset = IntervalDim + VelocityDim + NoteOnDim + NoteOffDim

EventDim = IntervalDim + VelocityDim + NoteOnDim + NoteOffDim + CCDim # 390

Time = constant_time_length

EmbeddingDim = embeding_size

HeadDim = 32
Heads = number_of_heads
ContextDim = HeadDim * Heads # 512

Layers = number_of_layers

In [ ]:
#@title Set and Print Created Hyperparameters
import numpy as np
import tensorflow as tf
from tensorflow.contrib.training import HParams

def default_hparams():
    return HParams(
        n_vocab=EventDim,
        n_ctx=ContextDim,
        n_embd=EmbeddingDim,
        n_head=Heads,
        n_layer=Layers,
        n_time=Time,
    )

hparams = default_hparams()
print(hparams)

### Load data from npz and converter to token sequence

In [ ]:
#@title Create a token sequence and plot the results
time_augmentation_delta = 0.15 #@param {type:"slider", min:0, max:1.2, step:0.01}
import numpy as np
import matplotlib.pyplot as plt
import librosa.display

def get_data(length=Time):
    index = np.random.randint(0, len(data_files))
    data = np.load(data_files[index])['eventlist']
    
    # time augmentation
    data[:, 0] *= np.random.uniform((1-time_augmentation_delta), (1+time_augmentation_delta))
    
    # absolute time to relative interval
    data[1:, 0] = data[1:, 0] - data[:-1, 0]
    data[0, 0] = 0
    
    # discretize interval into IntervalDim
    data[:, 0] = np.clip(np.round(data[:, 0] * IntervalDim), 0, IntervalDim - 1)
    
    # Note augmentation
    data[:, 2] += np.random.randint(-6, 6)
    data[:, 2] = np.clip(data[:, 2], 0, NoteOnDim - 1)
    
    eventlist = []
    for d in data:
        # append interval
        interval = d[0]
        eventlist.append(interval)
    
        # note on case
        if d[1] == 1:
            velocity = (d[3] / 128) * VelocityDim + VelocityOffset
            note = d[2] + NoteOnOffset
            eventlist.append(velocity)
            eventlist.append(note)
            
        # note off case
        elif d[1] == 0:
            note = d[2] + NoteOffOffset
            eventlist.append(note)
        # CC
        elif d[1] == 2:
            event = CCOffset + d[3]
            eventlist.append(event)
            
    eventlist = np.array(eventlist).astype(np.int)
    
    if len(eventlist) > (length+1):
        start_index = np.random.randint(0, len(eventlist) - (length+1))
        eventlist = eventlist[start_index:start_index+(length+1)]
        
    # pad zeros
    if len(eventlist) < (length+1):
        pad = (length+1) - len(eventlist)
        eventlist = np.pad(eventlist, (pad, 0), 'constant')
        
    x = eventlist[:length]
    y = eventlist[1:length+1]
    
    return x, y
    
x, y = get_data()
print('x shape : ', x.shape)
print('y shape : ', y.shape)
# print(x)
# print(y)
    
    
roll = np.zeros([len(x), EventDim])
for t, _x in enumerate(x):
    roll[t, _x] = 1

plt.figure(figsize=[18, 6])
librosa.display.specshow(roll.T)
plt.show()

### GPT-2 Model Setup and Initialization

In [ ]:
#@title GPT-2 source code from https://github.com/openai/gpt-2/blob/master/src/model.py
def shape_list(x):
    """Deal with dynamic shape in tensorflow cleanly."""
    static = x.shape.as_list()
    dynamic = tf.shape(x)
    return [dynamic[i] if s is None else s for i, s in enumerate(static)]

def softmax(x, axis=-1):
    x = x - tf.reduce_max(x, axis=axis, keepdims=True)
    ex = tf.exp(x)
    return ex / tf.reduce_sum(ex, axis=axis, keepdims=True)

def gelu(x):
    return 0.5*x*(1+tf.tanh(np.sqrt(2/np.pi)*(x+0.044715*tf.pow(x, 3))))

def norm(x, scope, *, axis=-1, epsilon=1e-5):
    """Normalize to mean = 0, std = 1, then do a diagonal affine transform."""
    with tf.variable_scope(scope):
        n_state = x.shape[-1].value
        g = tf.get_variable('g', [n_state], initializer=tf.constant_initializer(1))
        b = tf.get_variable('b', [n_state], initializer=tf.constant_initializer(0))
        u = tf.reduce_mean(x, axis=axis, keepdims=True)
        s = tf.reduce_mean(tf.square(x-u), axis=axis, keepdims=True)
        x = (x - u) * tf.rsqrt(s + epsilon)
        x = x*g + b
        return x

def split_states(x, n):
    """Reshape the last dimension of x into [n, x.shape[-1]/n]."""
    *start, m = shape_list(x)
    return tf.reshape(x, start + [n, m//n])

def merge_states(x):
    """Smash the last two dimensions of x into a single dimension."""
    *start, a, b = shape_list(x)
    return tf.reshape(x, start + [a*b])

def conv1d(x, scope, nf, *, w_init_stdev=0.02):
    with tf.variable_scope(scope):
        *start, nx = shape_list(x)
        w = tf.get_variable('w', [1, nx, nf], initializer=tf.random_normal_initializer(stddev=w_init_stdev))
        b = tf.get_variable('b', [nf], initializer=tf.constant_initializer(0))
        c = tf.reshape(tf.matmul(tf.reshape(x, [-1, nx]), tf.reshape(w, [-1, nf]))+b, start+[nf])
        return c

def attention_mask(nd, ns, *, dtype):
    """1's in the lower triangle, counting from the lower right corner.
    Same as tf.matrix_band_part(tf.ones([nd, ns]), -1, ns-nd), but doesn't produce garbage on TPUs.
    """
    i = tf.range(nd)[:,None]
    j = tf.range(ns)
    m = i >= j - ns + nd
    return tf.cast(m, dtype)

'''
MEMORY EFFICIENT IMPLEMENTATION OF RELATIVE POSITION-BASED ATTENTION
(Music Transformer, Cheng-Zhi Anna Huang et al. 2018)
'''
def attn(x, scope, n_state, *, hparams):
    assert x.shape.ndims == 3  # Should be [batch, sequence, features]
    assert n_state % hparams.n_head == 0

    def split_heads(x):
        # From [batch, sequence, features] to [batch, heads, sequence, features]
        return tf.transpose(split_states(x, hparams.n_head), [0, 2, 1, 3])

    def merge_heads(x):
        # Reverse of split_heads
        return merge_states(tf.transpose(x, [0, 2, 1, 3]))

    def mask_attn_weights(w):
        # w has shape [batch, heads, dst_sequence, src_sequence], where information flows from src to dst.
        _, _, nd, ns = shape_list(w)
        b = attention_mask(nd, ns, dtype=w.dtype)
        b = tf.reshape(b, [1, 1, nd, ns])
        w = w*b - tf.cast(1e10, w.dtype)*(1-b)
        return w
    
    def relative_attn(q):
        # q have shape [batch, heads, sequence, features]
        batch, heads, sequence, features = shape_list(q)
        E = tf.get_variable('E', [heads, sequence, features])
        # [heads, batch, sequence, features]
        q_ = tf.transpose(q, [1, 0, 2, 3])
        # [heads, batch * sequence, features]
        q_ = tf.reshape(q_, [heads, batch * sequence, features])
        # [heads, batch * sequence, sequence]
        rel = tf.matmul(q_, E, transpose_b=True)
        # [heads, batch, sequence, sequence]
        rel = tf.reshape(rel, [heads, batch, sequence, sequence])
        # [heads, batch, sequence, 1+sequence]
        rel = tf.pad(rel, ((0, 0), (0, 0), (0, 0), (1, 0)))
        # [heads, batch, sequence+1, sequence]
        rel = tf.reshape(rel, (heads, batch, sequence+1, sequence))
        # [heads, batch, sequence, sequence]
        rel = rel[:, :, 1:]
        # [batch, heads, sequence, sequence]
        rel = tf.transpose(rel, [1, 0, 2, 3])
        return rel
        
    def multihead_attn(q, k, v):
        # q, k, v have shape [batch, heads, sequence, features]
        w = tf.matmul(q, k, transpose_b=True)
        w = w + relative_attn(q)
        w = w * tf.rsqrt(tf.cast(v.shape[-1].value, w.dtype))

        w = mask_attn_weights(w)
        w = softmax(w)
        a = tf.matmul(w, v)
        return a

    with tf.variable_scope(scope):
        c = conv1d(x, 'c_attn', n_state*3)
        q, k, v = map(split_heads, tf.split(c, 3, axis=2))
        present = tf.stack([k, v], axis=1)

        a = multihead_attn(q, k, v)
        a = merge_heads(a)
        a = conv1d(a, 'c_proj', n_state)
        return a, present


def mlp(x, scope, n_state, *, hparams):
    with tf.variable_scope(scope):
        nx = x.shape[-1].value
        h = gelu(conv1d(x, 'c_fc', n_state))
        h2 = conv1d(h, 'c_proj', nx)
        return h2


def block(x, scope, *, hparams):
    with tf.variable_scope(scope):
        nx = x.shape[-1].value
        a, present = attn(norm(x, 'ln_1'), 'attn', nx, hparams=hparams)
        x = x + a
        m = mlp(norm(x, 'ln_2'), 'mlp', nx*4, hparams=hparams)
        x = x + m
        return x, present

def expand_tile(value, size):
    """Add a new axis of given size."""
    value = tf.convert_to_tensor(value, name='value')
    ndims = value.shape.ndims
    return tf.tile(tf.expand_dims(value, axis=0), [size] + [1]*ndims)

def model(hparams, X, scope='model', reuse=False):
    with tf.variable_scope(scope, reuse=reuse):
        results = {}
        batch, sequence = shape_list(X)

        wte = tf.get_variable('wte', [hparams.n_vocab, hparams.n_embd],
                             initializer=tf.random_normal_initializer(stddev=0.02))
        h = tf.gather(wte, X)

        # Transformer
        presents = []
        for layer in range(hparams.n_layer):
            h, present = block(h, 'h%d' % layer, hparams=hparams)
            presents.append(present)
        results['present'] = tf.stack(presents, axis=1)
        h = norm(h, 'ln_f')

        # Language model loss.  Do tokens <n predict token n?
        h_flat = tf.reshape(h, [batch*sequence, hparams.n_embd])
        logits = tf.matmul(h_flat, wte, transpose_b=True)
        logits = tf.reshape(logits, [batch, sequence, hparams.n_vocab])
        results['logits'] = logits
        return results

### Draw Main Graph

In [ ]:
#@title Create Model's Main Graph
hparams = default_hparams()
print(hparams)


tf.reset_default_graph()

X = tf.placeholder(tf.int32, [None, hparams.n_time])
Y = tf.placeholder(tf.int32, [None, hparams.n_time])

X_onehot = tf.one_hot(X, axis=2, depth=hparams.n_vocab)

logits = model(hparams, X)['logits']
probs = tf.nn.softmax(logits, axis=2)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits)
loss = tf.reduce_mean(cross_entropy)

#temperature = tf.Variable(1., name='temperature')
temperature = 0
u = tf.random.uniform(shape=tf.shape(logits[:, -1]), minval=1e-5, maxval=1.-1e-5)
u = (logits[:, -1] - tf.log(temperature + 1e-8)) - tf.log(-tf.log(u))
sample = tf.argmax(u, axis=1)

#dist = tf.distributions.Categorical(logits=logits[:, -1])
#sample = dist.sample()

'''
Train
'''
global_step = tf.Variable(0, name='global_step')
learning_rate = tf.Variable(1e-4, name='learning_rate')
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step)

'''
Session Open
'''


# GPU number to use
gpu_options = tf.GPUOptions(visible_device_list="0")
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

#config = tf.ConfigProto(device_count = {'GPU': 0})
#sess = tf.Session(config=config)

sess.run(tf.global_variables_initializer())

print('graph create')
#!nvidia-smi

### Load model if exist

In [ ]:
#@title Load existing Checkpoint or Intialize an empty model
import tensorflow.contrib.slim as slim
from tensorflow.python import pywrap_tensorflow

load_dir = 'save/gpt2-cc-interval100-attention2048-midi'
save_dir = 'save/gpt2-cc-interval100-attention2048-midi'

def get_variables_from_checkpoint_file(file_name):
    variables = []
    reader = pywrap_tensorflow.NewCheckpointReader(file_name)

    var_to_shape_map = reader.get_variable_to_shape_map()
    for key in sorted(var_to_shape_map):
        variables.append((key, var_to_shape_map[key]))

    return variables

saver = tf.train.Saver()

if True:
    restore_file = tf.train.latest_checkpoint(load_dir)
    if restore_file is not None:
        try:
            saver.restore(sess, restore_file)
            print("Model restored.", restore_file)
        except:
            saved_variables = get_variables_from_checkpoint_file(restore_file)
            model_variables = slim.get_variables_to_restore()
            restore_variables = []
            for model_variable in model_variables:
                for saved_variable_name, saved_variable_shape in saved_variables:
                    model_variable_name = model_variable.name.split(":")[0]
                    if saved_variable_name == model_variable_name and tuple(saved_variable_shape) == model_variable.shape:
                        restore_variables.append(model_variable)

            init_saver = tf.train.Saver(restore_variables)
            init_saver.restore(sess, restore_file)
            print("Model partially restored.")
    else:
        print('model not exist.')
        

### TensorboardX Logger

In [ ]:
#@title TensorboardX Logger
from tensorboardX import SummaryWriter

class Logger(SummaryWriter):
    def __init__(self, logdir):
        super(Logger, self).__init__(logdir)

    def log(self, log_string, value, iteration):
            self.add_scalar(log_string, value, iteration)
            
logger = Logger(save_dir)            

### Train Loop

In [ ]:
#@title Train the Model (From Checkpoint or Scratch). Recommended number of steps to achieve good results is 40k-60k. Do not forget to adjust this number according to the number of batches.
log_every_steps = 1000 #@param {type:"slider", min:0, max:1000, step:20}
save_every_steps = 1000 #@param {type:"slider", min:0, max:1000, step:20}
number_of_batches_per_step = 1 #@param {type:"slider", min:0, max:32, step:1}
learning_rate_cc = 0.0001 #@param {type:"slider", min:0, max:0.001, step:0.000001}
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import librosa.display
from time import sleep
import time

batch_size = number_of_batches_per_step # Works on Google P100 GPU and Colab Pro. Otherwise, use 1 or 2. Def = 1
log_every = log_every_steps # Display logs/plot data every indicated number of steps. Def = 10
save_every = save_every_steps # Save the trained model (checkpoint) every indicated number of steps. Def = 10

while(True):
    for _ in range(log_every_steps):
        _inputs = []
        _targets = []
        for _ in range(batch_size):
            while(True):
                x, y = get_data(hparams.n_time)
                if(x.shape == y.shape):
                    break
                 
            _inputs.append(x)
            _targets.append(y)
        _inputs = np.stack(_inputs)
        _targets = np.stack(_targets)
        print(_inputs.shape, _targets.shape)
        
        _, _global_step, _loss = sess.run([train_step, global_step, loss], 
                                          feed_dict={X: _inputs, 
                                                     Y: _targets,
                                                     learning_rate: learning_rate_cc})
        print(_global_step, _loss)
        
        if _global_step % log_every == 0:
            logger.log('loss', _loss, _global_step)
        
        if _global_step % save_every == 0:
            save_path = saver.save(sess, save_dir + '/checkpoint', global_step=_global_step)
            print("Model saved in path: %s" % save_path)
        
    clear_output()
    
    _inputs_onehot, _probs = sess.run([X_onehot, probs], feed_dict={X: _inputs})
    
    plt.figure(figsize=[18, 4])
    librosa.display.specshow(_inputs_onehot[0].T)
    plt.show()
    
    plt.figure(figsize=[18, 4])
    librosa.display.specshow(_probs[0].T)
    plt.show()

###(Optional) Save the last checkpoint/currently loaded model from memory to file.

In [ ]:
#@title Manual model save option from current memory
save_path = saver.save(sess, save_dir + '/checkpoint', global_step=_global_step)
print("Model saved in path: %s" % save_path)

### Generate Output Sequence and save it to a MIDI file.

In [ ]:
#@title WARNING: Created/resulting Model may produce (partially) plagiarized (overfitted) output. Excercise care and respect the copyright, please :) NOTE: Number of generated tokens must be >= Time Constant Hyperparameter of the model (i.e. if time_constant=2048, number_of_tokens_to_generate must be >= 2048). You can also manipulate other variables below to further improve generated output. Only the last batch is saved and displayed.
number_of_tokens_to_generate = 4096 #@param {type:"slider", min:128, max:8192, step:128}
number_of_batches = 4 #@param {type:"slider", min:0, max:32, step:1}
midi_ticks_multiplier = 5 #@param {type:"slider", min:0, max:64, step:1}
priming_sequence_delta = 4032 #@param {type:"slider", min:0, max:8192, step:64}
import librosa.display
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

import tensorflow.contrib.slim as slim
from tensorflow.python import pywrap_tensorflow

load_dir = 'save/gpt2-cc-interval100-attention2048-midi'
save_dir = 'save/gpt2-cc-interval100-attention2048-midi'

def get_variables_from_checkpoint_file(file_name):
    variables = []
    reader = pywrap_tensorflow.NewCheckpointReader(file_name)

    var_to_shape_map = reader.get_variable_to_shape_map()
    for key in sorted(var_to_shape_map):
        variables.append((key, var_to_shape_map[key]))

    return variables

saver = tf.train.Saver()

if True:
    restore_file = tf.train.latest_checkpoint(load_dir)
    if restore_file is not None:
        try:
            saver.restore(sess, restore_file)
            print("Model restored.", restore_file)
        except:
            saved_variables = get_variables_from_checkpoint_file(restore_file)
            model_variables = slim.get_variables_to_restore()
            restore_variables = []
            for model_variable in model_variables:
                for saved_variable_name, saved_variable_shape in saved_variables:
                    model_variable_name = model_variable.name.split(":")[0]
                    if saved_variable_name == model_variable_name and tuple(saved_variable_shape) == model_variable.shape:
                        restore_variables.append(model_variable)

            init_saver = tf.train.Saver(restore_variables)
            init_saver.restore(sess, restore_file)
            print("Model partially restored.")
    else:
        print('model not exist.')

N = number_of_tokens_to_generate # Number of steps/tokens you want to generate (for). Increasing over 2000 auto-switches Collab to CPU generation which is very slow. Def = 2048

x, y = get_data(hparams.n_time)
_inputs = np.zeros([number_of_batches, N], dtype=np.int32)
#_inputs[:(N-128), :len(x)] = x[None, :]
#_inputs[:, :len(x)] = x[None, :]
_inputs[:priming_sequence_delta, :len(x)] = x[None, :]
print(_inputs)
print('Priming Tokens Quantity: ')
print(len(x)-priming_sequence_delta)

for i in tqdm(range(N-Time)):

    _sample, _prob = sess.run([sample, probs], feed_dict={X: _inputs[:, i:i+Time]})
    _inputs[:, i+Time] = _sample 

print(_inputs.shape)

class Event():
    def __init__(self, time, note, cc, on, velocity):
        self.time = time
        self.note = note
        self.on = on
        self.cc = cc
        self.velocity = velocity

    def get_event_sequence(self):
        return [self.time, self.note, int(self.on)]

class Note():
    def __init__(self):
        self.pitch = 0
        self.start_time = 0
        self.end_time = 0

event_list = []
time = 0
event = None

EventDim = IntervalDim + VelocityDim + NoteOnDim + NoteOffDim # 388

for _input in _inputs[0]:
    # interval
    if _input < IntervalDim: 
        time += _input
        event = Event(time, 0, False, 0, 0)

    # velocity
    elif _input < NoteOnOffset:
        if event is None:
            continue
        event.velocity = (_input - VelocityOffset) / VelocityDim * 128
        #print('velocity : ', event.velocity)

    # note on
    elif _input < NoteOffOffset:
        if event is None:
            continue

        event.note = _input - NoteOnOffset
        event.on = True
        event_list.append(event)
        #event_list.append(Event(event.time + 100, event.note, False))
        event = None

    # note off
    elif _input < CCOffset:
        if event is None:
            continue
        event.note = _input - NoteOffOffset
        event.on = False
        event_list.append(event)
        event = None

    ## CC
    else:
        if event is None:
            continue
        event.cc = True
        on = _input - CCOffset == 1
        event.on = on
        #print(on)
        event_list.append(event)
        event = None

import midi
# Instantiate a MIDI Pattern (contains a list of tracks)
pattern = midi.Pattern()
# Instantiate a MIDI Track (contains a list of MIDI events)
track = midi.Track()
# Append the track to the pattern
pattern.append(track)

prev_time = 0
pitches = [None for _ in range(128)]
for event in event_list:
    tick = (event.time - prev_time) * midi_ticks_multiplier
    prev_time = event.time

    # case NOTE:
    if not event.cc:
        if event.on:
            if pitches[event.note] is not None:
                # Instantiate a MIDI note off event, append it to the track
                off = midi.NoteOffEvent(tick=0, pitch=event.note)
                track.append(off)
                pitches[event.note] = None

            # Instantiate a MIDI note on event, append it to the track
            on = midi.NoteOnEvent(tick=tick, velocity=int(event.velocity), pitch=event.note)
            track.append(on)
            pitches[event.note] = prev_time
        else:
            # Instantiate a MIDI note off event, append it to the track
            off = midi.NoteOffEvent(tick=tick, pitch=event.note)
            track.append(off)
            pitches[event.note] = None

    # case CC:
    elif event.cc:
        if event.on:
            cc = midi.ControlChangeEvent(tick=tick, control=64, value=64)
        else:
            cc = midi.ControlChangeEvent(tick=tick, control=64, value=0)

        track.append(cc)

    for pitch in range(128):
        if pitches[pitch] is not None and pitches[pitch] + 100 < prev_time:
            #print('here')
            off = midi.NoteOffEvent(tick=0, pitch=pitch)
            track.append(off)
            pitches[pitch] = None


# Add the end of track event, append it to the track
eot = midi.EndOfTrackEvent(tick=1)
track.append(eot)
# Print out the pattern
#print(pattern)
# Save the pattern to disk
midi.write_midifile("output_file.mid", pattern)

print('Successfully exported the output to output_file.mid')
print('Downloading output_file.mid')
from google.colab import files
files.download('/content/output_file.mid')

# III. Plot and Graph the Output

In [ ]:
#@title Plot/Graph the Output and Render Output for listening (WAV)
graphs_length_inches = 20 #@param {type:"slider", min:0, max:20, step:1}
notes_graph_height = 8 #@param {type:"slider", min:0, max:20, step:1}
rendered_wav_graph_height = 3 #@param {type:"slider", min:0, max:20, step:1}
# make the ticks absolute value
pattern.make_ticks_abs()
#print(pattern)

# create a note class
class Note:
    def __init__(self, pitch, start_time):
        self.pitch = pitch
        self.start_time = start_time
        self.end_time = None
        
    def __str__(self):
        return str(self.pitch) + ' ' + str(self.start_time) + ' ' + str(self.end_time)

# creating a sorted notelist
time_unit = pattern.resolution / 4

notelist = []
notes = [None for _ in range(128)]
for track in pattern:
    for event in track:
        if event.name == 'Note On':
            pitch = event.get_pitch()
            start_time = round(event.tick / time_unit)
            note = Note(pitch, start_time)
            notes[pitch] = note
        
        elif event.name == 'Note Off':
            pitch = event.get_pitch()
            note = notes[pitch]
            if note is not None:
                note.end_time = round(event.tick / time_unit)
                notelist.append(note)
                notes[pitch] = None

# Sort notelist by order of start_time and pitch
notelist = sorted(notelist, key=lambda note: (note.start_time, note.pitch))
                
#for note in notelist:
#    print(note)

# creating empty pianoroll
import numpy as np
from pypianoroll import Multitrack, Track
import matplotlib.pyplot as plt

length = notelist[-1].end_time
roll = np.zeros([int(length), 128])

track = Track(pianoroll=roll, program=0, is_drum=False, name='track')

#fig, ax = track.plot()
#fig.set_size_inches(18, 5)
#plt.show()

# Plot the output

for note in notelist:
    pitch = note.pitch    
    start_time = int(note.start_time)
    end_time = int(note.end_time)
    roll[start_time:end_time, pitch] = 1

track = Track(pianoroll=roll, program=0, is_drum=False, name='track')

fig, ax = track.plot()
fig.set_size_inches(graphs_length_inches, notes_graph_height)
plt.show()

# Generate rendering (WAV)

import pypianoroll as ppr
import pretty_midi

pm = ppr.Multitrack(tracks=[ppr.Track(roll)]).to_pretty_midi(20)
#audio = pm.fluidsynth()
audio = pm.synthesize()
#print(audio.shape)

plt.figure(figsize=[graphs_length_inches, rendered_wav_graph_height])
plt.plot(audio)
plt.show()

#import IPython.display as ipd
#ipd.Audio(audio, rate=16000)

###Save your model to your Google Drive directly as it is very fast :)

In [ ]:
#@title Standard Google Drive connect code
from google.colab import drive
drive.mount('/content/drive')